In [125]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

### (a) - data cleaning

Nothing to do.

### (b) - data completion

Nothing to do.

### (c) - reducing dimentions


It is obvious that **temp** (temperature in degrees Celsius) and **atemp**: (sensation of temperature in degrees Celsius) are correlated to each other. In addition, **temperature**, **humidity** and **windspeed** are correlating with categorical variable **weather**.

It could make sense to create a single variable out of **temp** and **atemp** instead of 2, but instead of guessing, we will performt presise feature selection later on, instead of taking guesses. Our data have 10 predictors,  which isn't that much, and plenty of training and testing data. It is not emperative to reduce dimentions. We will evaluate each predictor later on.


We have a 'year' predictor, and this might be a problem. Renting in one year might be significantly different from the rents in another year due to varios reasons: global events, such as pandemy, war, volcano eruption, etc...

Let's take a look at averages of renting by the year:

In [126]:
df_raw = pd.read_csv('./data/train.csv', sep=';')

def print_mean_and_std(year):
    filtered_df = df_raw[df_raw['year'] == year]
    print(f"{year} mean: {filtered_df['count'].mean()}, std: {filtered_df['count'].std()}")

print_mean_and_std(2011)
print_mean_and_std(2012)

2011 mean: 144.14694508894044, std: 134.36129950191017
2012 mean: 239.548031496063, std: 209.73105063880573


There is difference, but not someting unfincable. **We will remove year from our predictors**

### (d) - balancing data

Nothing to do.

### (e) - data normalization

In [127]:
# 1. Drop unnecessary columns
df = df_raw.drop(columns=['id', 'year'], axis=1)
df

,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count
0,23,3,0,0,2,23.78,27.275,73,11.0014,133
1,8,3,0,0,1,27.88,31.820,57,0.0000,132
2,2,1,0,1,1,20.50,24.240,59,0.0000,19
3,20,3,0,1,3,25.42,28.790,83,19.9995,58
4,17,3,0,1,3,26.24,28.790,89,0.0000,285
...,...,...,...,...,...,...,...,...,...,...
7684,18,1,0,1,1,13.94,15.150,42,22.0028,457
7685,3,1,0,1,1,9.02,11.365,51,11.0014,1
7686,15,2,0,0,1,21.32,25.000,19,27.9993,626
7687,19,4,0,1,1,12.30,14.395,45,15.0013,217


In [128]:
# Split data into features (X) and target (y)
X = df.drop('count', axis=1)
y = df['count'] 
X

,hour,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,23,3,0,0,2,23.78,27.275,73,11.0014
1,8,3,0,0,1,27.88,31.820,57,0.0000
2,2,1,0,1,1,20.50,24.240,59,0.0000
3,20,3,0,1,3,25.42,28.790,83,19.9995
4,17,3,0,1,3,26.24,28.790,89,0.0000
...,...,...,...,...,...,...,...,...,...
7684,18,1,0,1,1,13.94,15.150,42,22.0028
7685,3,1,0,1,1,9.02,11.365,51,11.0014
7686,15,2,0,0,1,21.32,25.000,19,27.9993
7687,19,4,0,1,1,12.30,14.395,45,15.0013


In [129]:
categorical_cols = ["season", "weather", "holiday", "workingday"]
non_categorical_cols =  X.columns.drop(categorical_cols)
non_categorical_cols

Index(['hour', 'temp', 'atemp', 'humidity', 'windspeed'], dtype='object')

In [130]:
X[non_categorical_cols]

,hour,temp,atemp,humidity,windspeed
0,23,23.78,27.275,73,11.0014
1,8,27.88,31.820,57,0.0000
2,2,20.50,24.240,59,0.0000
3,20,25.42,28.790,83,19.9995
4,17,26.24,28.790,89,0.0000
...,...,...,...,...,...
7684,18,13.94,15.150,42,22.0028
7685,3,9.02,11.365,51,11.0014
7686,15,21.32,25.000,19,27.9993
7687,19,12.30,14.395,45,15.0013


In [131]:
scaler = StandardScaler()
X_scaled  = scaler.fit_transform(X[non_categorical_cols])
X_scaled

array([[ 1.65352224,  0.44901756,  0.42031574,  0.58176099, -0.22017498],
       [-0.5155726 ,  0.97307586,  0.9541646 , -0.24721608, -1.56535958],
       [-1.38321054,  0.02977093,  0.0638292 , -0.14359394, -1.56535958],
       ...,
       [ 0.49667166,  0.13458259,  0.15309766, -2.2160366 ,  1.85822533],
       [ 1.07509695, -1.01834566, -1.0925497 , -0.86894887,  0.26890857],
       [ 1.3643096 ,  1.28751084,  1.31006385,  0.21908352, -0.70925854]])

In [132]:
# make a dataframe from the scaled data
X_scaled_df = pd.DataFrame(X_scaled, columns=non_categorical_cols)
X_scaled_df


,hour,temp,atemp,humidity,windspeed
0,1.653522,0.449018,0.420316,0.581761,-0.220175
1,-0.515573,0.973076,0.954165,-0.247216,-1.565360
2,-1.383211,0.029771,0.063829,-0.143594,-1.565360
3,1.219703,0.658641,0.598265,1.099872,0.880058
4,0.785884,0.763453,0.598265,1.410738,-1.565360
...,...,...,...,...,...
7684,0.930491,-0.808722,-1.003869,-1.024382,1.125010
7685,-1.238604,-1.437592,-1.448449,-0.558082,-0.220175
7686,0.496672,0.134583,0.153098,-2.216037,1.858225
7687,1.075097,-1.018346,-1.092550,-0.868949,0.268909


In [137]:
# combine the scaled data with the categorical columns
X_preprocessed = pd.concat([X_scaled_df, X[categorical_cols]], axis=1)
X_preprocessed

,hour,temp,atemp,humidity,windspeed,season,weather,holiday,workingday
0,1.653522,0.449018,0.420316,0.581761,-0.220175,3,2,0,0
1,-0.515573,0.973076,0.954165,-0.247216,-1.565360,3,1,0,0
2,-1.383211,0.029771,0.063829,-0.143594,-1.565360,1,1,0,1
3,1.219703,0.658641,0.598265,1.099872,0.880058,3,3,0,1
4,0.785884,0.763453,0.598265,1.410738,-1.565360,3,3,0,1
...,...,...,...,...,...,...,...,...,...
7684,0.930491,-0.808722,-1.003869,-1.024382,1.125010,1,1,0,1
7685,-1.238604,-1.437592,-1.448449,-0.558082,-0.220175,1,1,0,1
7686,0.496672,0.134583,0.153098,-2.216037,1.858225,2,1,0,0
7687,1.075097,-1.018346,-1.092550,-0.868949,0.268909,4,1,0,1


### (f) - Feature engineering

In [140]:
model = LinearRegression()

#  Perform stepwise forward selection
sfs = SequentialFeatureSelector(
    model,  
    n_features_to_select="auto",
    direction="forward",    
    scoring='neg_mean_squared_error'        
)

sfs.fit(X_preprocessed, y)
selected_features = sfs.get_support(indices=True)
selected_features, sfs.feature_names_in_[selected_features]

(array([0, 1, 3, 5], dtype=int64),
 array(['hour', 'temp', 'humidity', 'season'], dtype=object))

Our best model seems to contain predictors: 'hour', 'temp', 'humidity', 'season'

# (4)

In [148]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=1)

# (5) - regression

In [149]:
# linear regression with cross-validation
from sklearn.model_selection import cross_val_score
import numpy as np
 
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
np.mean(-scores), scores



(21664.868129868904,
 array([-22515.90909277, -20949.46097137, -19670.21957534, -22274.730062  ,
        -22914.02094787]))

In [150]:
# 2. Train a linear regression model
model.fit(X_train, y_train)
# 3. evaluate the model
s = model.score(X_test, y_test)
s

0.3250153162840016

In [ ]:
model2 = LinearRegression()
model2.fit(X, y)
